### ספריות


In [1]:
import os
import sys
import pandas as pd
from shutil import copy
from openpyxl import load_workbook
# from adding_an_addition_following_the_index import adding_an_addition
# from functions import change_Muni_Heb_to_Muni_Eng
# from division_into_traffic_zones_of_plans import division_into_traffic_zones
# from export_geo_layer_for_client_control import export_geo_layer
# from export_index_layer_for_client_control import export_index_layer
# from forecast import clientTaz
# from geographical_features import add_geographical_Features
# from index_layer import index_layer_fun
# from status_exists_for_control import export_status_exists
# from uploading_index_table_elements import uploading_index_table
# from uploading_index_table_elements import uploading_index_table
# from export_forecast_2040 import export_forecast
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


In [2]:
file_date=pd.Timestamp.today().strftime('%y%m%d')

### העלת משתנים להרצת הקוד


In [3]:
cwd = os.getcwd()

create_forecast_basic_ad_hoc_location = r'{}\create_forecast_basic_ad_hoc'.format(cwd)

sys.path.append(r'{}\current'.format(create_forecast_basic_ad_hoc_location))
sys.path.append(r'{}\arab_and_palestinian'.format(create_forecast_basic_ad_hoc_location))

df_inputs_outputs = pd.read_excel(r'{}\inputs_outputs.xlsx'.format(cwd))

### פונקציות


In [4]:
from global_functions import make_point, up_load_gdb, up_load_shp, up_load_df, find_files_with_pattern, make_point, up_load_gdb, up_load_shp

In [5]:
# מיקום לקוח
client_data_folder_location = df_inputs_outputs['location'][0]

# שם תרחיש (forecast_version)
forecast_version = df_inputs_outputs['location'][1]

# v_date
v_date = df_inputs_outputs['location'][2]

# מיקום פלט תחזית בסיס לפי גירסא
forecast_version_basic_folder_location=df_inputs_outputs['location'][4]

# מספר גירסא
forecast_version_number = df_inputs_outputs['location'][1]

index_file_name='index_format_for_creating_forecast_jtmt_input_{}_{}'.format(forecast_version,v_date)

In [6]:
client_data_folder_location

'C:\\Users\\dpere\\Documents\\JTMT\\Projects\\תחזיות_דמוגרפיות\\קבצי עבודה\\142_מתחם_אנגל\\בהת'

### העלת אזורי תנועה לחישוב


In [7]:
# נתיב תיקייה
folder_path=r'{}\For_approval\Reference_tabels\shp'.format(client_data_folder_location)
gpd_name='tochnit_check.gdb'

forecast = up_load_gdb(r'{}\{}'.format(folder_path,gpd_name),'TAZ_211028_V3_Published_with_client_changes')

### הוספת מאפיינים גיאוגרפים לאזורי תנועה


In [8]:
forecast_point = make_point(forecast)

# Load data layers
DISTRICT = up_load_gdb(
    r'{}\background_files\GIS_jtmt_forcast_v_3_2_Published.gdb'.format(cwd), 'TAZ_V3_2_220123_DISTRICT')
urban = up_load_gdb(
    r'{}\background_files\GIS_jtmt_forcast_v_3_2_Published.gdb'.format(cwd), 'TAZ_V3_2_220123_urban')
SCHOOLDISTRICT = up_load_gdb(
    r'{}\background_files\GIS_jtmt_forcast_v_3_2_Published.gdb'.format(cwd), 'TAZ_V3_2_220123_SCHOOLDISTRICT')
PUMA = up_load_gdb(
    r'{}\background_files\GIS_jtmt_forcast_v_3_2_Published.gdb'.format(cwd), 'TAZ_V3_2_220123_PUMA')
jerusalem_city = up_load_gdb(
    r'{}\background_files\GIS_jtmt_forcast_v_3_2_Published.gdb'.format(cwd), 'TAZ_V3_2_220123_In_jerusal')
subdistrict_il = up_load_gdb(
    r'{}\background_files\subdistrict2008.gdb'.format(cwd), 'subdistrict2008_ITM')
muni_JTMT = up_load_gdb(
    r'{}\background_files\MUNI_border.gdb'.format(cwd), 'muni_under_JTMT_ITM')
    
# מחפש קבצים של jeru_metro_jtmt_border
jeru_metro_jtmt_border_files = find_files_with_pattern(r'{}\background_files'.format(cwd), 'jeru_metro_jtmt_border')
filepath = ''

# לוקח את הקובץ של השכבה
for string in jeru_metro_jtmt_border_files:
    if string.endswith('.shp'):
        filepath = string

# מעלה את השכבה
jeru_metro_jtmt_border = up_load_shp(filepath)

# Geographical join between traffic zones and data layers
forecast_point_DISTRICT = forecast_point.sjoin(
    DISTRICT)[['Taz_num', 'puma2040_csv_DISTRICT']]
forecast_point_urban = forecast_point.sjoin(
    urban)[['Taz_num', 'BaseProjections2040_csv_urban']]
forecast_point_SCHOOLDISTRICT = forecast_point.sjoin(
    SCHOOLDISTRICT)[['Taz_num', 'puma2040_csv_SCHOOLDISTRICT']]
forecast_point_PUMA = forecast_point.sjoin(
    PUMA)[['Taz_num', 'puma2040_csv_PUMA']]
forecast_point_jerusalem_city = forecast_point.sjoin(
    jerusalem_city)[['Taz_num', 'jerusalem_city']]
forecast_point_subdistrict_il = forecast_point.sjoin(
    subdistrict_il[['geometry', 'ENG_NAME_nafa']])[['Taz_num', 'ENG_NAME_nafa']]
forecast_point_muni_JTMT = forecast_point.query('main_sector!="Palestinian"').sjoin(
    muni_JTMT[['Muni_Heb', 'Sug_Muni', 'CR_PNIM', 'geometry']])[['Taz_num', 'Muni_Heb', 'Sug_Muni', 'CR_PNIM']]
forecast_point_jeru_metro_jtmt_border = forecast_point.sjoin(
    jeru_metro_jtmt_border)[['Taz_num', 'jeru_metro']]

# Merge tables into one table
forecast = (forecast
            .merge(forecast_point_subdistrict_il, on='Taz_num', how='left')
            .merge(forecast_point_muni_JTMT, on='Taz_num', how='left')
            .merge(forecast_point_jeru_metro_jtmt_border, on='Taz_num', how='left')
            .merge(forecast_point_DISTRICT, on='Taz_num', how='left')
            .merge(forecast_point_urban, on='Taz_num', how='left')
            .merge(forecast_point_SCHOOLDISTRICT, on='Taz_num', how='left')
            .merge(forecast_point_PUMA, on='Taz_num', how='left')
            .merge(forecast_point_jerusalem_city, on='Taz_num', how='left'))

# Rename columns# Rename columns with flipped names
forecast.rename(columns={'ENG_NAME_nafa': 'zonetype'}, inplace=True)
forecast.rename(columns={'puma2040_csv_DISTRICT': 'DISTRICT'}, inplace=True)
forecast.rename(columns={'BaseProjections2040_csv_urban': 'urban'}, inplace=True)
forecast.rename(columns={'puma2040_csv_PUMA': 'PUMA'}, inplace=True)
forecast.rename(columns={'puma2040_csv_SCHOOLDISTRICT': 'SCHOOLDISTRICT'}, inplace=True)


# Data processing for feature columns
forecast.loc[forecast['main_sector'] == 'Palestinian', 'zonetype'] = 'Palestinian'
forecast['in_jerusalem_metropolin'] = 1
forecast.loc[forecast['jeru_metro'] == 0, 'in_jerusalem_metropolin'] = 0
forecast['yosh'] = 0
forecast.loc[forecast['zonetype'] == 'Judea and Samaria', 'yosh'] = 1

# Add constant columns
forecast['REGION'] = 1
forecast['slop'] = 0

forecast = forecast.set_index('Taz_num')
forecast['Taz_num'] = forecast.index

### שינוי עמודה Muni_Heb


In [9]:
muni_english = pd.read_excel(r'{}\background_files\english_names.xlsx'.format(cwd))
muni_heb_mapping = muni_english.set_index('Muni_Heb')

forecast = forecast.merge(muni_heb_mapping, how='left', left_on='Muni_Heb', right_index=True)
forecast.dropna(subset=['Muni_Heb'], inplace=True)

### ייצוא שכבת אזורי תנועה לבקרת לקוח


In [10]:
save_shp_path=r'{}\For_approval\{}_taz_for_approval.shp'.format(client_data_folder_location,file_date)
    
col=['Taz_num','Name_hebre', 'main_sector', 'classification_name','Student_by_Classrooms','geometry']

forecast[col].to_file(save_shp_path,index=False,encoding='UTF-8')

C:\Users\dpere\AppData\Local\Temp\ipykernel_14780\248974576.py:5: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  forecast[col].to_file(save_shp_path,index=False,encoding='UTF-8')


### מצב קיים לבקרה


In [11]:
# sys.path.insert(0, r'{}'.format(create_forecast_basic_location))
from run_forecast_basic_ad_hoc import run_notebook

notebook_path=r'{}\current\run_forecast_basic_ad_hoc.ipynb'.format(create_forecast_basic_ad_hoc_location)
folder_path=r'{}\For_approval\Reference_tabels'.format(client_data_folder_location)
pattern='TAZ_V'
matching_files=find_files_with_pattern(r'{}\shp'.format(folder_path), pattern)

forecast_2020=None

# אם יש שכבות חדשות
if len(matching_files) > 0:
        suffix = '.shp'
        filtered_files_strings = []
        for string in matching_files:
            if string.endswith(suffix):                    
                 filtered_files_strings.append(string)

        new_layer_path=filtered_files_strings[0]

        # #load excel file
        # workbook = load_workbook(filename=r"{}\inputs_outputs.xlsx".format(create_forecast_basic_location))

        # #open workbook
        # sheet = workbook.active

        # #modify the desired cell
        # sheet["B4"] = folder_path
        # sheet["B5"] = True
        # sheet["B6"] = filepath

        # #save the file
        # workbook.save(filename=r"{}\inputs_outputs.xlsx".format(create_forecast_basic_location))
        # מחליף את הנתיב הנוכחי לתיקייה current
        os.chdir(r'{}\current'.format(create_forecast_basic_ad_hoc_location))

        # להריץ את קוד בסיס עם השכבות
        params = {"folder_path": folder_path, "new_layer_path": new_layer_path}

        # הרצת המחברת עם פרמטרים
        updated_notebook = run_notebook(notebook_path, params)

#         execution_result = run_notebook(r'{}\run_basic_from_ad_hoc.ipynb'.format(create_forecast_basic_location))
#         print("ad-hoc-Notebook execution result:", execution_result)

#         if execution_result == True:
#             src_path = r'{}\forecast_2020_{}.xlsx'.format(folder_path, file_date)
#             destination_path = r'{}\forecast_2020_{}_with_taz_changes.xlsx'.format(folder_path, file_date)

#             copy(src_path, destination_path)
#             os.remove(src_path)

#             src_full_path = r'{}\2020_jtmt_forcast_full_{}.xlsx'.format(folder_path, file_date)
#             destination_full_path = r'{}\2020_jtmt_forcast_full_{}_with_taz_changes.xlsx'.format(folder_path, file_date)

#             copy(src_full_path, destination_full_path)
#             os.remove(src_full_path)

#             forecast_2020=up_load_df(r'{}'.format(folder_path), r'2020_jtmt_forcast_full_{}_with_taz_changes'.format(file_date))
#             #  forecast_2020=up_load_df(r'{}\background_files'.format(software_data_folder_location),'2020_jtmt_forcast_full_230720')

#             col=[]

#             col_20=['Taz_num','Taz_name',
#                 'main_secto',
#                 'aprt_20', 'pop_without_dorms_yeshiva',
#                 'student_toddlers',
#                 'student_gov',
#                 'cbs_muni_student_left_by_pre_of_demand_left',
#                 'uni_students', 'student_dorms',
#                 'emp_from_uni_student',
#                 'student_yeshiva',
#                 'emp_okev',
#                 'emp_not_okev','student']
                 
#             forecast_2020=pd.merge(forecast[col].reset_index(),forecast_2020[col_20],how='left',on='Taz_num').fillna(0)

#             save_excel_path=r'{}\For_approval\{}_forecast_2020_For_approval.xlsx'.format(client_data_folder_location,file_date)

#             forecast_2020[col_20].to_excel(save_excel_path,index=False)
#             # return forecast_2020[col_20]


#     # אם אין שכבות חדשות
# else:
#     #load excel file
#     workbook = load_workbook(filename=r"{}\inputs_outputs.xlsx".format(create_forecast_basic_location))

#     #open workbook
#     sheet = workbook.active

#     #modify the desired cell
#     sheet["B4"] = forecast_version_basic_folder_location
#     sheet["B5"] = False
#     sheet["B6"] = r'{}\background_files\TAZ_V4_230518_Published.shp'.format(create_forecast_basic_location)

#     #save the file
#     workbook.save(filename=r"{}\inputs_outputs.xlsx".format(create_forecast_basic_location))

#     forecast_2020=up_load_df(r'{}\background_files'.format(software_data_folder_location),'2020_jtmt_forcast_full_230720')

#     col=[]

#     col_20=['Taz_num','Taz_name',
#         'main_secto',
#         'aprt_20', 'pop_without_dorms_yeshiva',
#         'student_toddlers',
#         'student_gov',
#         'cbs_muni_student_left_by_pre_of_demand_left',
#         'uni_students', 'student_dorms',
#         'emp_from_uni_student',
#         'student_yeshiva',
#         'emp_okev',
#         'emp_not_okev','student']

#     forecast_2020=pd.merge(forecast[col].reset_index(),forecast_2020[col_20],how='left',on='Taz_num').fillna(0)

#     save_excel_path=r'{}\For_approval\{}_forecast_2020_For_approval.xlsx'.format(client_data_folder_location,file_date)

#     forecast_2020[col_20].to_excel(save_excel_path,index=False)

In [12]:
# forecast.head()